In [8]:
import pandas as pd
import requests
from PIL import Image
import re
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

In [9]:
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.58s/it]


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [11]:
# df = pd.read_excel("behaviour_simulation_train.xlsx")
name = "content_simulation_test_company.xlsx"
df = pd.read_excel(name)

In [12]:
df.head()

,id,date,likes,username,media,inferred company
0,1,2019-05-10 11:44:04,2,UPMCnews,[Photo(previewUrl='https://pbs.twimg.com/media...,upmc
1,2,2019-09-03 14:00:01,1,rbcgamnews,[Photo(previewUrl='https://pbs.twimg.com/media...,rbc
2,3,2018-12-02 05:51:16,502,BBCSport,[Photo(previewUrl='https://pbs.twimg.com/media...,bbc
3,4,2018-08-04 20:03:17,151,BBCSport,[Photo(previewUrl='https://pbs.twimg.com/media...,bbc
4,5,2019-02-11 04:29:29,362,uclagymnastics,[Photo(previewUrl='https://pbs.twimg.com/media...,ucla


In [13]:

def Captioning_image(df, media, model , start , end):
  Captions = []
  prompt = ""

  for num, text in enumerate(df.media[start:end]) :
    url_pattern = re.compile(r"'(https?://[^\s]+)'")
    urls = url_pattern.findall(text)
    caption = ""
    for i, url in enumerate(urls):
      try :
        if 'large' in url or 'Large' in url or "LARGE" in url:
          image = Image.open(requests.get(url, stream=True).raw).convert('RGB')
          inputs = processor(image, text=prompt, return_tensors="pt").to(device, torch.float16).to(device)
          generated_ids = model.generate(**inputs, max_new_tokens=25)
          generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
          caption += f"Image{i} : {generated_text}. "
      except :
        caption += "Error while doing image captioning."
    Captions.append(caption)
    if num%25 == 0: 
      print(num) 
  return Captions


In [14]:
start_list = [x for x in range(0,5001,5000)]
end_list = [x for x in range(5000,10001,5000)]
for start,end in zip(start_list,end_list):
    captions = Captioning_image(df, "media", model , start , end)
    caption_df = pd.DataFrame(captions)
    file_path = name + "_" + str(start)+"-"+str(end)+".csv"
    caption_df.to_csv(file_path, index=False)

0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
2050
2075
2100
2125
2150
2175
2200
2225
2250
2275
2300
2325
2350
2375
2400
2425
2450
2475
2500
2525
2550
2575
2600
2625
2650
2675
2700
2725
2750
2775
2800
2825
2850
2875
2900
2925
2950
2975
3000
3025
3050
3075
3100
3125
3150
3175
3200
3225
3250
3275
3300
3325
3350
3375
3400
3425
3450
3475
3500
3525
3550
3575
3600
3625
3650
3675
3700
3725
3750
3775
3800
3825
3850
3875
3900
3925
3950
3975
4000
4025
4050
4075
4100
4125
4150
4175
4200
4225
4250
4275
4300
4325
4350
4375
4400
4425
4450
4475
4500
4525
4550
4575
4600
4625
4650
4675
4700
4725
4750
4775
4800
4825
4850
4875
4900
4925
4950
4975
0
25
50
75
100
125
150
175
200
225
250
275
30

/home/imroze/Vaibhav/Inter IIT Tech Meet 12.0/MP4/myvenv/lib/python3.10/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


2200
2225
2250
2275
2300
2325
2350
2375
2400
2425
2450
2475
2500
2525
2550
2575
2600
2625
2650
2675
2700
2725
2750
2775
2800
2825
2850
2875
2900
2925
2950
2975
3000
3025
3050
3075
3100
3125
3150
3175
3200
3225
3250
3275
3300
3325
3350
3375
3400
3425
3450
3475
3500
3525
3550
3575
3600
3625
3650
3675
3700
3725
3750
3775
3800
3825
3850
3875
3900
3925
3950
3975
4000
4025
4050
4075
4100
4125
4150
4175
4200
4225
4250
4275
4300
4325
4350
4375
4400
4425
4450
4475
4500
4525
4550
4575
4600
4625
4650
4675
4700
4725
4750
4775
4800
4825
4850
4875
4900
4925
4950
4975
